In [1]:
import pandas as pd
from lib.pitch_control import plot_pitch_control, KNNPitchControl, SpearmanPitchControl, FernandezPitchControl
from lib.draw import pitch
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm

In [2]:
df = pd.read_csv('data/all_goals.csv', low_memory=False)
df.edgecolor.fillna('white', inplace=True)
df.bgcolor.fillna('black', inplace=True)
df.sample(5)

,bgcolor,dx,dy,edgecolor,frame,play,player,player_num,team,x,y,z,coords,player_obj,num,name,play_id
43737,red,-0.092968,0.101735,white,72,Liverpool [2] - 0 Salzburg,5050,10.0,attack,39.644576,34.977421,0.0,NaN,NaN,NaN,NaN,12
65674,darkred,-0.110462,-0.000057,black,148,Bournemouth 0 - 3 Liverpool,12087,NaN,defense,17.460213,45.818722,0.0,NaN,NaN,NaN,NaN,2
60568,blue,-0.340664,-0.107330,white,141,Liverpool [2] - 0 Everton,11936,NaN,defense,27.500474,21.028281,0.0,NaN,NaN,NaN,NaN,9
69895,yellow,-0.272864,-0.069455,black,184,Liverpool [1] - 0 Watford,10098,NaN,defense,24.774833,33.745271,0.0,NaN,NaN,NaN,NaN,7
59900,red,-0.180742,0.042659,white,47,Liverpool [2] - 0 Everton,2131,NaN,attack,80.943165,69.480600,0.0,NaN,NaN,NaN,NaN,9


In [3]:
df.play.unique()

array(['Liverpool [3] - 0 Bournemouth', 'Bayern 0 - [1] Liverpool',
       'Fulham 0 - [1] Liverpool', 'Southampton 1 - [2] Liverpool',
       'Liverpool [2] - 0 Porto', 'Porto 0 - [2] Liverpool',
       'Liverpool [4] - 0 Barcelona', 'Liverpool [1] - 0 Wolves',
       'Liverpool [3] - 0 Norwich', 'Liverpool [2] - 1 Chelsea',
       'Liverpool [2] - 1 Newcastle', 'Liverpool [2] - 0 Salzburg',
       'Genk 0 - [3] Liverpool', 'Liverpool [2] - 0 Man City',
       'Liverpool [1] - 0 Everton', 'Liverpool [2] - 0 Everton',
       'Bournemouth 0 - 3 Liverpool', 'Liverpool [1] - 0 Watford',
       'Leicester 0 - [3] Liverpool', 'Barcelona 1 - [2] Real Madrid'],
      dtype=object)

In [4]:
knn_pitch_control_voronoi = KNNPitchControl()
knn_pitch_control_spearman = KNNPitchControl(lags=[0, 10, 20], smoothing=8)
knn_pitch_control_fernandez = KNNPitchControl(lags=[0, 10, 20], smoothing=6, distance_basis=250)
spearman_pitch_control = SpearmanPitchControl()
fernandez_pitch_control = FernandezPitchControl()

In [5]:
# Initialize results file
import os
results_file_path = 'res/results.csv'
grids_file_path = 'res/grids.csv'
if os.path.isfile(results_file_path):
    df_results = pd.read_csv(results_file_path)
    csv_results = open(results_file_path, 'a')
    csv_grids = open(grids_file_path, 'a')
else:
    df_results = pd.DataFrame(columns=['play_frame_id'])
    csv_results = open(results_file_path, 'w')
    csv_results.write('play_frame_id,model,play,frame,inference_time\n')
    csv_grids = open(grids_file_path, 'w')
    csv_grids.write('play_frame_id,model_name' + ''.join(map(lambda x: x, [f',c{i}' for i in range(7314)])) + '\n')

In [6]:
def close_and_reopen_file(csv, filename):
    csv.close()
    return open(filename, 'a')

In [7]:
def run_model(csv_results, csv_grids, model, model_name, play_frame_id, df_frame, df_play=None):
    t0 = time()
    if df_play is None:
        model.grid['control'] = model.predict(df_frame).round(2)
    else:
        model.grid['control'] = model.predict(df_frame, df_play).round(2)
    inference_time = time() - t0
    
    model.grid['play_frame_id'] = play_frame_id
    model.grid['model'] = model_name
    
    csv_results.write(f'{play_frame_id},{model_name},{play},{frame_no},{inference_time}\n')
    csv_grids.write(f'{play_frame_id},{model_name}' + ''.join(map(lambda x: x, [f',{i}' for i in model.grid['control'].tolist()])) + '\n')

In [8]:
play_frame_id = 0
for play in df.play.unique()[[4, 5, -1]]:
    print(play)
    df_play = df[df.play == play]
    for frame_no in tqdm(df_play.frame.unique()):
        play_frame_id += 1
        if play_frame_id in df_results.play_frame_id.unique():
            continue

        df_frame = df_play[df_play.frame == frame_no]
        
        run_model(csv_results, csv_grids, knn_pitch_control_voronoi, 'KNN (Voronoi)', play_frame_id, df_frame)
        run_model(csv_results, csv_grids, knn_pitch_control_spearman, 'KNN (Spearman)', play_frame_id, df_frame)
        run_model(csv_results, csv_grids, knn_pitch_control_fernandez, 'KNN (Fernandez)', play_frame_id, df_frame)
        run_model(csv_results, csv_grids, spearman_pitch_control, 'Spearman Pitch Control', play_frame_id, df_frame)
        run_model(csv_results, csv_grids, fernandez_pitch_control, 'Fernandez Pitch Control', play_frame_id, df_frame, df_play)

        csv_results = close_and_reopen_file(csv_results, results_file_path)
        csv_grids = close_and_reopen_file(csv_grids, grids_file_path)

  0%|          | 0/257 [00:00<?, ?it/s]

Liverpool [2] - 0 Porto
Porto 0 - [2] Liverpool


 75%|███████▌  | 193/257 [02:12<02:03,  1.93s/it]/home/liaad/Desktop/KNNPitchControl/lib/pitch_control.py:387: RuntimeWarning: overflow encountered in exp
  self.grid['control'] = 1 - (1 / (1 + np.exp(-Za + Zh))).flatten()
 75%|███████▌  | 194/257 [02:35<02:54,  2.77s/it]/home/liaad/Desktop/KNNPitchControl/lib/pitch_control.py:387: RuntimeWarning: overflow encountered in exp
  self.grid['control'] = 1 - (1 / (1 + np.exp(-Za + Zh))).flatten()
 76%|███████▌  | 195/257 [02:57<03:57,  3.83s/it]/home/liaad/Desktop/KNNPitchControl/lib/pitch_control.py:387: RuntimeWarning: overflow encountered in exp
  self.grid['control'] = 1 - (1 / (1 + np.exp(-Za + Zh))).flatten()
 76%|███████▋  | 196/257 [03:20<05:14,  5.15s/it]/home/liaad/Desktop/KNNPitchControl/lib/pitch_control.py:387: RuntimeWarning: overflow encountered in exp
  self.grid['control'] = 1 - (1 / (1 + np.exp(-Za + Zh))).flatten()
 77%|███████▋  | 197/257 [03:40<06:33,  6.56s/it]/home/liaad/Desktop/KNNPitchControl/lib/pitch_control.py:38

Barcelona 1 - [2] Real Madrid


/home/liaad/Desktop/KNNPitchControl/lib/pitch_control.py:387: RuntimeWarning: overflow encountered in exp
  self.grid['control'] = 1 - (1 / (1 + np.exp(-Za + Zh))).flatten()
 50%|████▉     | 144/289 [35:53<39:46, 16.46s/it] /home/liaad/Desktop/KNNPitchControl/lib/pitch_control.py:387: RuntimeWarning: overflow encountered in exp
  self.grid['control'] = 1 - (1 / (1 + np.exp(-Za + Zh))).flatten()
 50%|█████     | 145/289 [36:10<39:34, 16.49s/it]/home/liaad/Desktop/KNNPitchControl/lib/pitch_control.py:387: RuntimeWarning: overflow encountered in exp
  self.grid['control'] = 1 - (1 / (1 + np.exp(-Za + Zh))).flatten()
 61%|██████    | 177/289 [43:42<23:33, 12.62s/it]/home/liaad/Desktop/KNNPitchControl/lib/pitch_control.py:387: RuntimeWarning: overflow encountered in exp
  self.grid['control'] = 1 - (1 / (1 + np.exp(-Za + Zh))).flatten()
 62%|██████▏   | 178/289 [43:54<23:16, 12.59s/it]/home/liaad/Desktop/KNNPitchControl/lib/pitch_control.py:387: RuntimeWarning: overflow encountered in exp
 

# Run Individual Models

In [9]:
knn_pitch_control_spearman_doublesmooth = KNNPitchControl(lags=[0, 10, 20], smoothing=[8, 10])

In [10]:
for play in df.play.unique()[[4, 5, -1]]:
    print(play)
    df_play = df[df.play == play]
    for frame_no in tqdm(df_play.frame.unique()):
        play_frame_id += 1

        df_frame = df_play[df_play.frame == frame_no]
        
        run_model(csv_results, csv_grids, knn_pitch_control_spearman_doublesmooth, 'KNN (Spearman 2 Smooth)', play_frame_id, df_frame)

        csv_results = close_and_reopen_file(csv_results, results_file_path)
        csv_grids = close_and_reopen_file(csv_grids, grids_file_path)

  0%|          | 0/195 [00:00<?, ?it/s]

Liverpool [2] - 0 Porto


  0%|          | 0/257 [00:00<?, ?it/s]

Porto 0 - [2] Liverpool


  0%|          | 0/289 [00:00<?, ?it/s]

Barcelona 1 - [2] Real Madrid


100%|██████████| 289/289 [02:50<00:00,  1.70it/s]
